# Introduction to AVISO-data

We will use the satellite-altimetry-derived ocean-surface current product AVISO-data as a dataset to validate the methods. The data is freely available from the [AVISO]((http://aviso.oceanobs.com)).

The sea surface height $ h $ acts as a stream function for the geostrophic ocean surface velocity field. Particle trajectories on the ocean surface are approximately trajectories of the 2D system of ODEs:

\begin{align}
\dot{\varphi} &= -\dfrac{g}{R^2f(\theta)\cos(\theta)}\partial_{\theta}h(\phi, \theta, t) \\
\dot{\theta} &= \dfrac{g}{R^2f(\theta)\cos(\theta)}\partial_{\varphi}h(\phi, \theta, t),
\end{align}

where $ \varphi, \theta $ respectively are the longitudinal and latitudinal position of the particle, $ g $ is the constant of gravity, $ R $ is the mean radius, $ f(\theta) = 2\Omega \sin(\theta) $ is the coriolis parameter and $ \Omega $ is the mean angular velocity of the earth.

The AVISO-data provides global coverage of ocean currents at weekly time-intervals with a spatial resolution of $ 0.25^{\circ} $. As part of this work, we focus on the commonly studied Agulhas region in the Southern Ocean:

\begin{equation}
(\varphi, \theta) \in [14^{\circ}W, 9^{\circ}E] \times [39^{\circ}S, 21^{\circ}S]
\end{equation}

over a period of (maximum) 90 days starting at $ t_0 = 24 $ November 2006. The area under study is characterized by intense eddy activity.

"Nan"-values in the velocity data correspond to points where the particles is not defined (e.g. points on land). The velocity at those points is zero.

# Add Folders to Path

We start by adding the necessary folders to the current working path.

In [1]:
%%time
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")

Wall time: 0 ns


# Import data

In [2]:
%%time
import scipy.io as sio

#Import velocity data from file in data-folder
mat_file = sio.loadmat('AVISO_raw.mat')

U = mat_file['u']
V = mat_file['v']
x = mat_file['x']
y = mat_file['y']
time_data = mat_file['t']

Wall time: 105 ms


# Interpolate in time using tricubic spline interpolation from [1]

In order to more accurately capture the velocity data in the open sea we employ the tricubic spline interpolation from Lekien  .The velocity data has a spatial resolution of 0.25 degrees, whereas the temporal resolution is originally seven days. We interpolate the data to a finer temporal resolution using spline interpolation. The new temporal resolution is 1 day.

NOTE: The package [pytricubic](#https://github.com/danielguterding/pytricubic) needs to be manually installed. The installation guidelines can be found [here](#https://github.com/danielguterding/pytricubic).

In [3]:
# import tricubic
import tricubic

# import progress bar
from tqdm.notebook import tqdm

# import numpy
import numpy as np

# import function to check location of the particle.
from ipynb.fs.defs.check_location import check_location

# new temporal resolution
new_dt = 1 # in units of days

# grid of original data
X_original, Y_original = np.meshgrid(x, y)
time_data_original = time_data.copy()

# spacing of grid-size of original data
dx_data = X_original[0,1]-X_original[0,0]
dy_data = Y_original[1,0]-Y_original[0,0]
dt_data = time_data[0, 1]-time_data[0, 0]

# create Interpolant object

# refined spatial/temporal grid
X_refined, Y_refined = np.meshgrid(np.arange(np.min(x), np.max(x)+0.25, 0.25), np.arange(np.min(y), np.max(y)+0.25, 0.25)) # 0.125 degrees
time_data_refined = np.arange(np.min(time_data_original), np.max(time_data_original)+new_dt, new_dt).reshape(1, -1)  # .1 day resolution

# Initialize arrays for velocities over refined grid
U_refined = np.zeros((X_refined.shape[0], X_refined.shape[1], time_data_refined.shape[1]))
V_refined = np.zeros((X_refined.shape[0], X_refined.shape[1], time_data_refined.shape[1]))

# defined domain
# defined_domain is an array which denotes point in the meshgrid where the velocity field is defined (=1) 
# and points where it is not defined (=0). 
# This is needed to evaluate whether the particle is in a cell with "Nan" values at its grid-points.
defined_domain = np.isfinite(U[:,:,0]).astype(int)

# transform nan values to '0' for interpolation
U[np.isnan(U)] = 0
V[np.isnan(V)] = 0

# Interpolant for u/v
Interpolant_u = tricubic.tricubic(list(U), [X_original.shape[0], X_original.shape[1], time_data_original.shape[1]])
Interpolant_v = tricubic.tricubic(list(V), [X_original.shape[0], X_original.shape[1], time_data_original.shape[1]])

# Iterate over grid

# iterate in y
for idx_y in tqdm(range(Y_refined.shape[0])):
    
    # iterate in x
    for idx_x in range(X_refined.shape[1]):
        
        # iterate in time
        for idx_t in range(time_data_refined.shape[1]):
            
            x0 = np.array([X_refined[idx_y, idx_x], Y_refined[idx_y, idx_x]])
            
            # check location of particle
            location, idx_x_loc, idx_y_loc = check_location(X_original, Y_original, defined_domain, x0)
            
            # If particle is in a grid-cell without "Nan"-values, then use interpolant to evaluate velocity field at refined grid-point.
            if location == "IN":
                
                x_scaled = (x0[0]-X_original[0, 0])/dx_data
                
                y_scaled = (x0[1]-Y_original[0, 0])/dy_data
                
                time_scaled = (time_data_refined[0,idx_t]-time_data_original[0, 0])/dt_data
            
                x_list = list(np.array([y_scaled, x_scaled, time_scaled]))
            
                U_refined[idx_y, idx_x, idx_t] = Interpolant_u.ip(x_list)
                V_refined[idx_y, idx_x, idx_t] = Interpolant_v.ip(x_list)
            
            else:
                
                U_refined[idx_y, idx_x, idx_t], V_refined[idx_y, idx_x, idx_t] = np.nan, np.nan

  0%|          | 0/105 [00:00<?, ?it/s]

In [4]:
# save data

mdic = {"u": U_refined, "v": V_refined, "x": X_refined[0,:], "y": Y_refined[:,0], "t": time_data_refined}
sio.savemat('AVISO.mat', mdic)

# References

[1] Lekien, F., & Marsden, J. (2005). Tricubic interpolation in three dimensions. International Journal for Numerical Methods in Engineering, 63(3), 455-471.